# Conditional Generative Adversarial Networks (cGANs)

**Learning Objectives**

- Build a Conditional GAN architecture (consisting of a generator and discriminator) in Keras 
- Define the loss for the generator and discriminator
- Define a training step for the GAN using tf.keras.Model subclassing
- Train the cGAN on the MNIST dataset

## Introduction
This notebook demonstrates how to build and train a [Conditional Generative Adversarial Network](https://arxiv.org/abs/1411.1784).

The basic GAN that we discussed in the previous lab([deepconv-gan.ipynb](./deepconv_gan.ipynb)) is trained in a completely unsupervised way on images that we want to learn how to generate. <br>
Latent representations, such as a random noise vector, are then used to explore and sample the learned image space.

A simple enhancement is to add an external flag to our inputs with a label.<br>
For instance, consider the MNIST dataset, which consists of handwritten digits from 0 to 9. 

Normally, the GAN just learns the distribution of digits, and when the generator is given random noise vectors, it generates different digits.

![Basic GAN](https://user-images.githubusercontent.com/6895245/171208971-3ff1c023-cdd7-47c2-b734-77e07b8eb6ac.png)

However, which digit are generated cannot be controlled.

During training, as with MNIST, we may know the actual digit or class designation for each image. <br>
That extra information can be included as a feature in our GAN training that can then be used at inference time. 

With conditional GANs (cGANs), image generation can be conditional on the digit, so we are able to home in on the specific digit of interest’s distribution. 

Then, at inference time we can create an image of a specific digit by passing in the desired digit instead of receiving a random digit.

![cGAN](https://user-images.githubusercontent.com/6895245/171209451-4b95c856-fafa-40b4-87b6-c7232c1de841.png)

In this notebook, you are going to learn how to build conditional GAN.

## Import TensorFlow and other libraries

In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (
    BatchNormalization,
    Concatenate,
    Conv2D,
    Conv2DTranspose,
    Dense,
    Embedding,
    Flatten,
    Input,
    LeakyReLU,
    Reshape,
)

## Constants

In [ ]:
IMAGE_SHAPE = (28, 28, 1)
NUM_CLASSES = 10
LATENT_DIM = 512
PROJECTION_SHAPE = (7, 7, 56)

## Create the generator and discriminator models for Conditional GAN

In the previous lab, vanilla GAN was defined using the [Keras Sequential API](https://www.tensorflow.org/guide/keras#sequential_model), but here we use [Keras Functional API](https://www.tensorflow.org/guide/keras/functional) to create more complicated models for Conditional GAN.

Firstly, we need to define layers to embed digit (0-9) information, which will be used both in Generator and Discriminator.<br>
The embedding should be a rank 3 tensor (height, width, 1) like an image so that it can be concatenated with the mnist image later.

**Exercise**: Complete `create_digit_embedding` function by adding:
- `Embedding` layer to embed class index into `embedding_dim` dimension.
- `Dense` layer to change the dimensionality of the embedding into num_pixels (image height * width) dimension.
- `Reshape` layer to reshape the output of the dense layer from 1D tensor to 2D tensor to concatenate with images later.

In [ ]:
def create_digit_embedding(digits, num_classes, embedding_dim, image_shape):
    # TODO: Define an Embedding layer.
    embedded_digits = Embedding(...)(digits)

    num_pixels = image_shape[0] * image_shape[1]  # height * width

    # TODO: Define a Dense layer.
    dense_digits = Dense(...)(embedded_digits)

    # TODO: Define a Reshape layer
    digits_image = Reshape(...)(dense_digits)

    return digits_image

Let's try to generate a digit embedding image. Since it is not trained yet it is just a random image.

In [ ]:
DIGIT_TO_EMBED = 0  # [0,9]

digit_embedding = create_digit_embedding(
    tf.convert_to_tensor([DIGIT_TO_EMBED]), NUM_CLASSES, 128, (7, 7, 1)
)[0]
plt.imshow(digit_embedding)

### The Generator

The generator of Conditional GAN creates images from random latent vectors and digit vectors.
It takes a noise vector in the `latent_dim` shape, and a digit index. Both vectors are embedded and reshaped to the same height and width defined in `projection_shape` to get concatenated. Then the generator generates 28x28x1 MNIST images from the concatenated rank 3 vector.

The generator model uses [Conv2DTranspose](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose) to upsample images usually along with [BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization). Also it uses [LeakyRelu](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LeakyReLU) for a nonlinear activation.

Now we define the function `create_dcgan_generator` to build a cGAN generator. This will:

1. map a latent vector to a rank-3 tensor. Here we use `projection_shape` variable to define the shape of a latent image.
2. map a digit index to rank-3 tensors (call `create_digit_embedding` function). The output Tensor shape is formatted to the same height and width as `projection_shape` to get concatenated with an image in the next step.
3. concatenate 1 and 2.
4. pass 3 to GAN generator that has blocks consisting of `Conv2DTranspose`, `BatchNormalization` and `LeakyRelu` layer. Note that this part is almost the same as the DCGAN generator from the previous notebook.
5. return a Model object.

**Exercise**: Complete `create_dcgan_generator` function:
- Complete `create_digit_embedding` function defined above.
- Add concatenate layer to stack `latent_image` and `digit_embeddings`.

In [ ]:
def create_dcgan_generator(latent_dim, num_classes, projection_shape):
    # Random vector input
    latent_vector = Input(name="latent_input", shape=(latent_dim,))

    num_image_pixel = (
        projection_shape[0] * projection_shape[1] * projection_shape[2]
    )

    x = Dense(name="latent_dense", units=num_image_pixel)(latent_vector)
    x = LeakyReLU(name="latent_leakyrelu", alpha=0.2)(x)

    latent_image = Reshape(
        name="latent_to_image_reshape", target_shape=projection_shape
    )(x)

    # Receive digit inputs ([0,9])
    digits = Input(name="digit_input", shape=())

    # TODO: Complete create_digit_embedding function
    # Project digits to rank 3 tensor digit embeddings.
    digit_embeddings = create_digit_embedding(...)

    # TODO: concatenate image and label here.
    concatenated_inputs = Concatenate(...)(...)

    x = Conv2DTranspose(128, 4, 2, "same")(concatenated_inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2DTranspose(128, 4, 2, "same")(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    outputs = Conv2DTranspose(
        1, 3, 1, "same", activation="tanh", name="generator_output"
    )(x)

    return tf.keras.Model(
        inputs=[latent_vector, digits], outputs=outputs, name="dcgan_generator"
    )

In [ ]:
dcgan_generator = create_dcgan_generator(
    latent_dim=LATENT_DIM,
    num_classes=NUM_CLASSES,
    projection_shape=PROJECTION_SHAPE,
)
dcgan_generator.summary()

Let's plot the generator model and check how it is built.

In [ ]:
# Plot generator model.
tf.keras.utils.plot_model(
    model=dcgan_generator, show_shapes=True, show_layer_names=True
)

### The Discriminator

Next, we will build the discriminator. The discriminator is a CNN-based image classifier. <br>
It should take in an image of shape 28x28x1 and a digit index that will be embedded and reshaped to 28x28x1 as well, concatenate them and return a single classification indicating if that image is real or not.

Let's define the function `create_dcgan_discriminator` to build a cGAN discriminator. This will:

1. receive an image input in `image_shape` shape, which can be either real or fake.
2. map a digit index to rank-3 tensors (call `create_digit_embedding` function). The output Tensor shape is formatted to the same height and width as `image_shape` to get concatenated with an image in the next step.
3. concatenate 1 and 2
4. pass 3 to GAN generator that has blocks consisting of `Conv2D`, `BatchNormalization` and `LeakyRelu` layer. Note that this part is almost the same as the DCGAN discriminator from the previous notebook.
5. return a Model object


**Exercise**: Complete the entire `create_dcgan_discriminator` function following the steps above.

In [ ]:
def create_dcgan_discriminator(image_shape, num_classes):
    # TODO: Complete the function.

    return tf.keras.Model(
        inputs=[images, digits], outputs=outputs, name="dcgan_discriminator"
    )

In [ ]:
dcgan_discriminator = create_dcgan_discriminator(
    image_shape=IMAGE_SHAPE, num_classes=NUM_CLASSES
)
dcgan_discriminator.summary()

In [ ]:
# Plot discriminator model.
tf.keras.utils.plot_model(
    model=dcgan_discriminator, show_shapes=True, show_layer_names=True
)

We'll create a custom `Model` class named `GAN` that we can use to facilitate training of our generator and discriminator networks.

## Define Training Loop

In GAN training, since discriminators and generators have different optimization goals, we train these two models "alternately". <br>
Training a model alternately in this case means, when we train the discriminator, the generator's parameters are frozen and not updated. And then, we train the generator while freezing the discriminator.


So here, let's define separate training steps for the two models.

### Discriminator Training Steps

In the `discriminator_train_step` function below, we define a discriminator specific training routine that includes steps to:
1. sample random points in the latent space.
2. generate images from noise from random vectors created in step 1.
3. create training features by combining generated images with real images.
4. create labels for generated images (0) and real images (1). Here we apply **one-sided label smoothing** to real labels. Refer to the description of one-sided label smoothing below
5. call the discriminator to get predictions and compute loss in a `tf.GradientTape` context, 
6. get gradients of the parameters of the discriminator from `GradientTape` and update them. Note that the generator is not updated in the discriminator training step.

**one-sided label smoothing**: To multiply real labels (1) by a float constant in the range [0.0, 1.0] to help the discriminator avoid becoming overconfident in its predictions based on only a small set of features within the images, which the generator may then exploit (causing it to become good at beating the discriminator but not at image generation).

**Exercise**: Complete the discriminator training step.
- Add a line to call the `generator` f|unction.
- Create fake labels(`fake_labels`) and real_labels(`real_labels`) using `tf.zeros` and `tf.ones`.
- Apply one-sided label smoothing to the real labels.

In [ ]:
def discriminator_train_step(
    discriminator,
    generator,
    discriminator_optimizer,
    loss_fn,
    discriminator_accuracy,
    one_sided_label_smoothing,
    batch_size,
    real_images,
    input_digits,
):
    # Sample random points in the latent space.
    random_latent_vectors = tf.random.normal(shape=(batch_size, LATENT_DIM))

    # Generate images from noise.
    # TODO: complete generator function
    generated_images = generator(...)

    # Combine generated images with real images.
    combined_images = tf.concat(values=[generated_images, real_images], axis=0)

    # Create fake and real labels.
    # TODO: Define labels.
    fake_labels = ...
    real_labels = ...

    # Smooth real labels to help with training.
    # TODO: Apply one-side smoothing.
    real_labels = ...

    # Combine labels to be in-line with combined images.
    labels = tf.concat(values=[fake_labels, real_labels], axis=0)

    # Train ONLY the discriminator.
    with tf.GradientTape() as tape:
        predictions = discriminator(
            inputs=[
                combined_images,
                tf.concat(values=[input_digits, input_digits], axis=0),
            ]
        )
        discriminator_loss = loss_fn(y_true=labels, y_pred=predictions)

    pred_prob = tf.math.sigmoid(predictions)
    discriminator_accuracy.update_state(y_true=labels, y_pred=pred_prob)

    grads = tape.gradient(
        target=discriminator_loss,
        sources=discriminator.trainable_weights,
    )

    discriminator_optimizer.apply_gradients(
        grads_and_vars=zip(grads, discriminator.trainable_weights)
    )

    return discriminator_loss, discriminator_accuracy.result()

### Generator Training Steps

In the same way, we define the generator specific training steps in `generator_train_step` function that includes steps to:
1. sample random points in the latent space.
2. create labels for generated images (1) as if they're real images.
3. in a `tf.GradientTape` context, call the generator to generate images, pass them to the discriminator, and compute loss using labels created in step 2.
4. get gradients of the parameters of the generator from `GradientTape` and update them. Note that the discriminator is not updated in the discriminator training step.



**Exercise**: Define the generator training steps following the instruction above and the discriminator training process.

In [ ]:
def generator_train_step(
    discriminator,
    generator,
    generator_optimizer,
    loss_fn,
    batch_size,
    input_digits,
):
    # Sample random points in the latent space.
    random_latent_vectors = tf.random.normal(shape=(batch_size, LATENT_DIM))

    # Create labels as if they're real images.
    labels = tf.ones(shape=(batch_size, 1))

    # TODO: Complete the training process.
    # Call generator and discriminator, and compute the generator loss.
    with tf.GradientTape() as tape:
        ...

    # TODO: get gradients for generator weights.
    # Train ONLY the generator.
    grads = tape.gradient(...)

    generator_optimizer.apply_gradients(
        grads_and_vars=zip(grads, generator.trainable_weights)
    )

    return generator_loss

### Outer Training Loop
Now both training loops are defined.<br>
Let's wrap them in a `train_step` function and define an outer training loop.

Here we call `discriminator_train_step` and `generator_train_step` in order to train a discriminator and a generator sequentially, and return two losses of two models.

In [ ]:
DISCRIMINATOR_OPTIMIZER = tf.keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.999, epsilon=1e-8
)
GENERATOR_OPTIMIZER = tf.keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.999, epsilon=1e-8
)

LOSS_FN = tf.keras.losses.BinaryCrossentropy(from_logits=True)

ONE_SIDED_LABEL_SMOOTHING = 0.9

DISCRIMINATOR_ACCURACY = tf.keras.metrics.BinaryAccuracy()


def train_step(features):
    real_images = features["images"]
    input_digits = features["digits"]

    # Get dynamic batch size to use for random noise vectors.
    batch_size = tf.shape(input=real_images)[0]

    # First train discriminator.
    discriminator_loss, discriminator_accuracy_value = discriminator_train_step(
        dcgan_discriminator,
        dcgan_generator,
        DISCRIMINATOR_OPTIMIZER,
        LOSS_FN,
        DISCRIMINATOR_ACCURACY,
        ONE_SIDED_LABEL_SMOOTHING,
        batch_size,
        real_images,
        input_digits,
    )

    # Then train generator.
    generator_loss = generator_train_step(
        dcgan_discriminator,
        dcgan_generator,
        GENERATOR_OPTIMIZER,
        LOSS_FN,
        batch_size,
        input_digits,
    )
    losses = {
        "discriminator_loss": discriminator_loss,
        "discriminator_accuracy": discriminator_accuracy_value,
        "generator_loss": generator_loss,
    }

    return losses

### Create input data pipeline with tf.data
Now our model is ready. Let's create a data pipeline with tf data. <br>
Note that we have to pass not only images but also labels to CGAN.


In [ ]:
def create_train_dataset(batch_size):
    # Get and format MNIST data.
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    # Let's combine both training and test sets.
    combined_images = np.concatenate([x_train, x_test])
    combined_images = (combined_images.astype("float32") - 127.5) / 255.0
    combined_images = np.reshape(combined_images, newshape=(-1, 28, 28, 1))

    combined_labels = np.concatenate([y_train, y_test])

    # Create tf.data.Dataset for training.
    dataset = tf.data.Dataset.from_tensor_slices(
        tensors={"images": combined_images, "digits": combined_labels}
    )
    dataset = dataset.shuffle(buffer_size=10000).batch(
        batch_size=batch_size, drop_remainder=True
    )

    return dataset

In [ ]:
BATCH_SIZE = 128
EPOCHS = 10  # Change numbers for better generation

training_dataset = create_train_dataset(batch_size=BATCH_SIZE)

dataset_length = training_dataset.cardinality().numpy()

## Train CGAN
We can train a model by calling the `train_step` function in a loop.<br>
It takes around 7 minutes with a GPU (NVIDIA T4) to train 10 epochs.

In [ ]:
output_keys = ["discriminator_loss", "discriminator_accuracy", "generator_loss"]

history = {key: [] for key in output_keys}

for epoch in range(EPOCHS):
    print(f"{'=' * 20} EPOCH {epoch + 1} {'=' * 20}")
    epoch_history = {key: [] for key in output_keys}

    for i, features in enumerate(training_dataset, 1):
        losses = train_step(features)
        for key in output_keys:
            epoch_history[key].append(losses[key])

        if i % 100 == 0 or i == dataset_length:
            performance_str = ", ".join(
                [
                    f"{key}: {tf.math.reduce_mean(epoch_history[key]):5f}"
                    for key in output_keys
                ]
            )
            print(f"step {i:3d} / {dataset_length}: {performance_str}")

    for key in output_keys:
        history[key].append(tf.math.reduce_mean(epoch_history[key]))

    DISCRIMINATOR_ACCURACY.reset_state()

Let's plot the loss history and some generated images using our trained model.

In [ ]:
def plot_loss_history(history):
    plt.plot(history["discriminator_loss"])
    plt.plot(history["discriminator_accuracy"])
    plt.plot(history["generator_loss"])
    plt.title("Training losses")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend(output_keys, loc="upper left")
    plt.show()

In [ ]:
plot_loss_history(history=history)

How does the loss curve look?

Remember, our discriminator and generator are trying to optimize different loss values and battling each other.<br>
So if you see one loss is going down and the other is going up steeply during training, you can understand something is wrong with the balance between the two models. Then you should try to tune the optimization strategy, learning rate scheduler, or model architecture itself to balance these two.

Also, it would be useful to monitor the discriminator accuracy.<br>
If it is close to 50%, that means the discriminator cannot classify real or fake anymore. With that, we can understand that our generator is doing a very good job of creating realistic images, or the discriminator does not fit at all.

## Generate images
Now our CGAN is ready to generate images. Note that we can specify which number we want our generator to generate.

In [ ]:
LABEL_TO_GENERATE = 9  # 0-9
latent_vectors = tf.random.normal(shape=(1, LATENT_DIM))

generated_image = dcgan_generator(
    inputs=[latent_vectors, tf.constant([LABEL_TO_GENERATE], dtype=tf.int32)]
)
plt.imshow(
    tf.reshape(generated_image, generated_image.shape[1:-1]), cmap="gray"
)

Let's try to generate multiple numbers.

This is a helper function to visualize multiple images.

In [ ]:
def plot_images(images):
    """Plots images.

    Args:
        images: np.array, array of images of
            [num_images, image_size, image_size, num_channels].
    """
    num_images = len(images)

    plt.figure(figsize=(20, 20))
    for i in range(num_images):
        image = images[i]
        plt.subplot(1, num_images, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(tf.reshape(image, image.shape[:-1]), cmap="gray")
    plt.show()

In [ ]:
num_examples = 20
latent_vectors = tf.random.normal(shape=(num_examples, LATENT_DIM))
for i in range(10):
    labels = tf.ones(shape=num_examples, dtype=tf.int32) * i
    plot_images(images=dcgan_generator(inputs=[latent_vectors, labels]))

Looks good! You can try to get better images by specifying larger epochs (like 50).


## (Optional) Define GAN class
Instead of defining separated training loops, we can also combine them into a class by subclassing [tf.keras.Model](https://www.tensorflow.org/api_docs/python/tf/keras/Model)

By defining our own model class, we can use the standard model APIs like `model.fit()` by defining the original training steps and `model.save()` to save a model object in TensorFlow saved model format easily. Also, we can override the functions of tf.keras.Model to change behaviors of our model.

In addition to the training functions defined above, here we add these functions in our Conditional_GAN class:
- `__init__()`: instantiates model class. Here we pass the generator and discriminator that we defined above. Also, note that we specify `one_sided_label_smoothing` constant here that is used in discriminator training.
- `compile()`: overrides the `Model.compile()` function to be able to define optimizers of a generator and a discriminator separately, and to add a loss function that is used in both models.
- `metrics()`: defines metrics of the model. By adding some metrics in this `metrics` property so that `reset_states()` can be called automatically at the start of each epoch or at the start of `evaluate()`.


In [ ]:
class Conditional_GAN(tf.keras.Model):
    def __init__(
        self, discriminator, generator, latent_dim, one_sided_label_smoothing
    ):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.one_sided_label_smoothing = one_sided_label_smoothing

    def compile(
        self,
        discriminator_optimizer,
        generator_optimizer,
        loss_fn,
        discriminator_accuracy,
    ):
        super().compile()
        self.discriminator_optimizer = discriminator_optimizer
        self.generator_optimizer = generator_optimizer
        self.loss_fn = loss_fn
        self.discriminator_accuracy = discriminator_accuracy

    def discriminator_train_step(self, batch_size, real_images, input_digits):
        # reuse the function defined above
        return discriminator_train_step(
            self.discriminator,
            self.generator,
            self.discriminator_optimizer,
            self.loss_fn,
            self.discriminator_accuracy,
            self.one_sided_label_smoothing,
            batch_size,
            real_images,
            input_digits,
        )

    def generator_train_step(self, batch_size, input_digits):
        # reuse the function defined above
        return generator_train_step(
            self.discriminator,
            self.generator,
            self.generator_optimizer,
            self.loss_fn,
            batch_size,
            input_digits,
        )

    def train_step(self, features):
        real_images = features["images"]
        input_digits = features["digits"]

        # Get dynamic batch size to use for random noise vectors.
        batch_size = tf.shape(input=real_images)[0]

        # First train discriminator.
        (
            discriminator_loss,
            discriminator_accuracy_value,
        ) = self.discriminator_train_step(batch_size, real_images, input_digits)

        # Then train generator.
        generator_loss = self.generator_train_step(batch_size, input_digits)
        losses = {
            "discriminator_loss": discriminator_loss,
            "discriminator_accuracy": discriminator_accuracy_value,
            "generator_loss": generator_loss,
        }
        return losses

    @property
    def metrics(self):
        return [self.discriminator_accuracy]

We can instantiate the Conditional GAN model and compile it.

Please note that instantiaion calls `__init__` function and complilation does `compile` function of `Conditional_GAN` class.

In [ ]:
dcgan_generator = create_dcgan_generator(
    latent_dim=LATENT_DIM,
    num_classes=NUM_CLASSES,
    projection_shape=PROJECTION_SHAPE,
)

dcgan_discriminator = create_dcgan_discriminator(
    image_shape=IMAGE_SHAPE, num_classes=NUM_CLASSES
)

# Instantiate a Conditional_GAN instance using our DCGAN generator and discriminator.
dcgan = Conditional_GAN(
    discriminator=dcgan_discriminator,
    generator=dcgan_generator,
    latent_dim=LATENT_DIM,
    one_sided_label_smoothing=0.9,
)

dcgan.compile(
    discriminator_optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.0002, beta_1=0.5, beta_2=0.999, epsilon=1e-8
    ),
    generator_optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.0002, beta_1=0.5, beta_2=0.999, epsilon=1e-8
    ),
    loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    discriminator_accuracy=tf.keras.metrics.BinaryAccuracy(),
)

## Train CGAN
We can train a model by simply calling `model.fit()`. <br>
It take around 7 minutes with a GPU (NVIDIA T4) to train 10 epochs.

In [ ]:
BATCH_SIZE = 128
EPOCHS = 10  # Change numbers for better generation

# Train DCGAN model.
dcgan_history = dcgan.fit(
    create_train_dataset(batch_size=BATCH_SIZE), epochs=EPOCHS
)

In [ ]:
plot_loss_history(history=dcgan_history.history)

In [ ]:
num_examples = 20
latent_vectors = tf.random.normal(shape=(num_examples, LATENT_DIM))
for i in range(10):
    labels = tf.ones(shape=num_examples, dtype=tf.int32) * i
    plot_images(images=dcgan_generator(inputs=[latent_vectors, labels]))


## Summary
In this notebook, we learned how to build and train a Conditional GAN model.


Copyright 2022 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License